# Deep Learning

## Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.


The goal of this assignment is to explore regularization techniques.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in notmist.ipynb.

In [56]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.


In [57]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


### Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

# Logistic Regression 

In [17]:
train_subset = 10000
graph = tf.Graph()
with graph.as_default():
    #input data
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset,weights) + biases
    reg_beta = 0.1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + reg_beta* tf.nn.l2_loss(weights)) 
    
    
    #Optimizer
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.25).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights) + biases)
    
    

lets run it

In [18]:
num_steps = 801
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(801):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                    predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 315.913208
Training accuracy: 8.3%
Validation accuracy: 9.7%
Test accuracy: 9.1%
Loss at step 100: 2.758106
Training accuracy: 80.2%
Validation accuracy: 79.0%
Test accuracy: 86.2%
Loss at step 200: 0.998930
Training accuracy: 81.8%
Validation accuracy: 80.5%
Test accuracy: 87.4%
Loss at step 300: 0.986638
Training accuracy: 81.8%
Validation accuracy: 80.5%
Test accuracy: 87.5%
Loss at step 400: 0.985645
Training accuracy: 81.8%
Validation accuracy: 80.5%
Test accuracy: 87.5%
Loss at step 500: 0.985180
Training accuracy: 81.8%
Validation accuracy: 80.5%
Test accuracy: 87.5%
Loss at step 600: 0.984937
Training accuracy: 81.9%
Validation accuracy: 80.6%
Test accuracy: 87.5%
Loss at step 700: 0.984805
Training accuracy: 81.9%
Validation accuracy: 80.6%
Test accuracy: 87.5%
Loss at step 800: 0.984731
Training accuracy: 81.9%
Validation accuracy: 80.5%
Test accuracy: 87.5%


# Neural Network With Hidden Layer

In [36]:
batch_size = 128
train_subset = 10000
graph = tf.Graph()
hidden_layer = 1024
with graph.as_default():
    #input data
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(np.float32,shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer]))
    biases_hidden = tf.Variable(tf.truncated_normal([hidden_layer]))
    weights = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden) + biases_hidden)
    logits = tf.matmul(logits_hidden,weights) + biases
    reg_beta = 0.001
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + reg_beta* tf.nn.l2_loss(weights_hidden) + reg_beta * tf.nn.l2_loss(weights)) 
    
    
    #Optimizer
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+ biases_hidden), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+ biases_hidden), weights) + biases)

In [39]:
num_steps = 3001

with tf.Session(graph = graph) as session :
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.  
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss,train_prediction],feed_dict = feed_dict)
        
        if step % 500 == 0:
            print("Minibatch Loss as step %d : %f" % (step, l))
            print("Minibatch Accuracy : %.lf%%" % accuracy(predictions,batch_labels))
            print("Validation Accuracy : %.lf%%" % accuracy(valid_prediction.eval(),valid_labels))
            print("Test Accuracy : %.lf%%" % accuracy(test_prediction.eval(),test_labels))
        

Initialized
Minibatch Loss as step 0 : 622.306702
Minibatch Accuracy : 6%
Validation Accuracy : 21%
Test Accuracy : 23%
Minibatch Loss as step 500 : 197.749237
Minibatch Accuracy : 84%
Validation Accuracy : 81%
Test Accuracy : 87%
Minibatch Loss as step 1000 : 114.518021
Minibatch Accuracy : 81%
Validation Accuracy : 81%
Test Accuracy : 89%
Minibatch Loss as step 1500 : 68.359032
Minibatch Accuracy : 91%
Validation Accuracy : 82%
Test Accuracy : 89%
Minibatch Loss as step 2000 : 41.192886
Minibatch Accuracy : 89%
Validation Accuracy : 85%
Test Accuracy : 92%
Minibatch Loss as step 2500 : 25.106058
Minibatch Accuracy : 91%
Validation Accuracy : 86%
Test Accuracy : 92%
Minibatch Loss as step 3000 : 15.500160
Minibatch Accuracy : 88%
Validation Accuracy : 86%
Test Accuracy : 93%


# Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [60]:
train_subset = 1000
num_steps = 3001
train_dataset_overfit = train_dataset[0:train_subset+1,:]
train_labels_overfit = train_labels[0:train_subset+1,:]
with tf.Session(graph = graph) as session :
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_overfit.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_overfit[offset:(offset + batch_size), :]
        batch_labels = train_labels_overfit[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.  
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss,train_prediction],feed_dict = feed_dict)
        
        if step % 500 == 0:
            print("Minibatch Loss as step %d : %f" % (step, l))
            print("Minibatch Accuracy : %.lf%%" % accuracy(predictions,batch_labels))
            print("Validation Accuracy : %.lf%%" % accuracy(valid_prediction.eval(),valid_labels))
            print("Test Accuracy : %.lf%%" % accuracy(test_prediction.eval(),test_labels))
        

Initialized
Minibatch Loss as step 0 : 602.929321
Minibatch Accuracy : 9%
Validation Accuracy : 38%
Test Accuracy : 42%
Minibatch Loss as step 500 : 190.193207
Minibatch Accuracy : 100%
Validation Accuracy : 77%
Test Accuracy : 84%
Minibatch Loss as step 1000 : 115.343567
Minibatch Accuracy : 100%
Validation Accuracy : 77%
Test Accuracy : 84%
Minibatch Loss as step 1500 : 69.950844
Minibatch Accuracy : 100%
Validation Accuracy : 77%
Test Accuracy : 84%
Minibatch Loss as step 2000 : 42.422249
Minibatch Accuracy : 100%
Validation Accuracy : 76%
Test Accuracy : 84%
Minibatch Loss as step 2500 : 25.728029
Minibatch Accuracy : 100%
Validation Accuracy : 77%
Test Accuracy : 84%
Minibatch Loss as step 3000 : 15.605494
Minibatch Accuracy : 100%
Validation Accuracy : 77%
Test Accuracy : 85%


# Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?

In [64]:
batch_size = 128
train_subset = 10000
graph = tf.Graph()
hidden_layer = 1024
with graph.as_default():
    #input data
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(np.float32,shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer]))
    biases_hidden = tf.Variable(tf.truncated_normal([hidden_layer]))
    weights = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden) + biases_hidden)
    logits_dropout = tf.matmul(tf.nn.dropout(logits_hidden,0.5),weights) + biases
    logits = tf.matmul(logits_hidden,weights) + biases
    reg_beta = 0.001
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_dropout,tf_train_labels) + reg_beta* tf.nn.l2_loss(weights_hidden) + reg_beta * tf.nn.l2_loss(weights)) 
    
    
    #Optimizer
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+ biases_hidden), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+ biases_hidden), weights) + biases)

In [65]:
num_steps = 3001

with tf.Session(graph = graph) as session :
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.  
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss,train_prediction],feed_dict = feed_dict)
        
        if step % 500 == 0:
            print("Minibatch Loss as step %d : %f" % (step, l))
            print("Minibatch Accuracy : %.lf%%" % accuracy(predictions,batch_labels))
            print("Validation Accuracy : %.lf%%" % accuracy(valid_prediction.eval(),valid_labels))
            print("Test Accuracy : %.lf%%" % accuracy(test_prediction.eval(),test_labels))
        

Initialized
Minibatch Loss as step 0 : 777.305420
Minibatch Accuracy : 14%
Validation Accuracy : 24%
Test Accuracy : 26%
Minibatch Loss as step 500 : 204.859985
Minibatch Accuracy : 81%
Validation Accuracy : 81%
Test Accuracy : 87%
Minibatch Loss as step 1000 : 116.302757
Minibatch Accuracy : 83%
Validation Accuracy : 80%
Test Accuracy : 88%
Minibatch Loss as step 1500 : 69.382439
Minibatch Accuracy : 88%
Validation Accuracy : 82%
Test Accuracy : 89%
Minibatch Loss as step 2000 : 41.442474
Minibatch Accuracy : 88%
Validation Accuracy : 84%
Test Accuracy : 90%
Minibatch Loss as step 2500 : 25.358868
Minibatch Accuracy : 84%
Validation Accuracy : 84%
Test Accuracy : 91%
Minibatch Loss as step 3000 : 15.585005
Minibatch Accuracy : 86%
Validation Accuracy : 85%
Test Accuracy : 92%


# Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
One avenue you can explore is to add multiple layers.
Another one is to use learning rate decay:
`global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)`

In [75]:
batch_size = 128
train_subset = 10000
graph = tf.Graph()
hidden_layer = 1024
with graph.as_default():
    #input data
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(np.float32,shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_layer]))
    biases_hidden = tf.Variable(tf.truncated_normal([hidden_layer]))
    weights = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden) + biases_hidden)
    logits_dropout = tf.matmul(tf.nn.dropout(logits_hidden,0.5),weights) + biases
    logits = tf.matmul(logits_hidden,weights) + biases
    reg_beta = 0.00
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_dropout,tf_train_labels) + reg_beta* tf.nn.l2_loss(weights_hidden) + reg_beta * tf.nn.l2_loss(weights)) 
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5,global_step,decay_steps=500,decay_rate=0.9)
    #Optimizer
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+ biases_hidden), weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+ biases_hidden), weights) + biases)

In [77]:
num_steps = 10001

with tf.Session(graph = graph) as session :
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.  
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss,train_prediction],feed_dict = feed_dict)
        
        if step % 500 == 0:
            print("Minibatch Loss as step %d : %f" % (step, l))
            print("Minibatch Accuracy : %.lf%%" % accuracy(predictions,batch_labels))
            print("Validation Accuracy : %.lf%%" % accuracy(valid_prediction.eval(),valid_labels))
            print("Test Accuracy : %.lf%%" % accuracy(test_prediction.eval(),test_labels))
        

Initialized
Minibatch Loss as step 0 : 497.256409
Minibatch Accuracy : 8%
Validation Accuracy : 24%
Test Accuracy : 26%
Minibatch Loss as step 500 : 39.136353
Minibatch Accuracy : 80%
Validation Accuracy : 78%
Test Accuracy : 85%
Minibatch Loss as step 1000 : 19.836321
Minibatch Accuracy : 78%
Validation Accuracy : 81%
Test Accuracy : 87%
Minibatch Loss as step 1500 : 12.477496
Minibatch Accuracy : 84%
Validation Accuracy : 81%
Test Accuracy : 88%
Minibatch Loss as step 2000 : 4.958841
Minibatch Accuracy : 87%
Validation Accuracy : 81%
Test Accuracy : 88%
Minibatch Loss as step 2500 : 4.081308
Minibatch Accuracy : 81%
Validation Accuracy : 81%
Test Accuracy : 88%
Minibatch Loss as step 3000 : 2.724838
Minibatch Accuracy : 81%
Validation Accuracy : 81%
Test Accuracy : 88%
Minibatch Loss as step 3500 : 6.158775
Minibatch Accuracy : 81%
Validation Accuracy : 81%
Test Accuracy : 88%
Minibatch Loss as step 4000 : 5.730414
Minibatch Accuracy : 85%
Validation Accuracy : 81%
Test Accuracy : 88